# Model using bigger data

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

## Get data

In [2]:
df = pd.read_csv('../raw_data/diabetic_data.csv')
df.shape

(101766, 50)

## Clean data

In [3]:
# Check data types
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [4]:
# Checking for missing values in the dataset
# In the dataset, missing values are represented as '?' sign

missing_percentages = df.apply(lambda col: (col == '?').mean() * 100 if col.dtype == object else 0)
for col, percentage in missing_percentages.items():
    if percentage > 0:
        print(f"{col}: {percentage:.2f}%")


race: 2.23%
weight: 96.86%
payer_code: 39.56%
medical_specialty: 49.08%
diag_1: 0.02%
diag_2: 0.35%
diag_3: 1.40%


In [5]:
df.isna().mean() * 100

encounter_id                 0.000000
patient_nbr                  0.000000
race                         0.000000
gender                       0.000000
age                          0.000000
weight                       0.000000
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                   0.000000
medical_specialty            0.000000
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.000000
diag_2                       0.000000
diag_3                       0.000000
number_diagnoses             0.000000
max_glu_serum               94.746772
A1Cresult                   83.277322
metformin                    0.000000
repaglinide                  0.000000
nateglinide 

In [6]:
df.gender.value_counts()

gender
Female             54708
Male               47055
Unknown/Invalid        3
Name: count, dtype: int64

In [7]:
# Dealing with missing data
## weight: 96.86% -> drop
## payer_code: 39.56% -> drop
## medical_specialty: 49.08% -> drop

#dropping columns with large number of missing values
df = df.drop(['weight','payer_code','medical_specialty'], axis = 1)

# Dropping rows with few missing values
df = df[df['diag_1'] != '?']
df = df[df['diag_2'] != '?']
df = df[df['diag_3'] != '?']
df = df[df['race'] != '?']
df = df[df['gender'] != 'Unknown/Invalid']
df = df[df['discharge_disposition_id'] != 11]

# Replace values in A1Cresult and max_glu_serum
a1c_replacements = {'>7': 1, '>8': 1, 'Norm': 0}
df['A1Cresult'] = df['A1Cresult'].replace(a1c_replacements).fillna(-99)

max_glu_serum_replacements = {'>200': 1, '>300': 1, 'Norm': 0}
df['max_glu_serum'] = df['max_glu_serum'].replace(max_glu_serum_replacements).fillna(-99)

/tmp/ipykernel_39916/1239617468.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['A1Cresult'] = df['A1Cresult'].replace(a1c_replacements).fillna(-99)
/tmp/ipykernel_39916/1239617468.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['max_glu_serum'] = df['max_glu_serum'].replace(max_glu_serum_replacements).fillna(-99)


In [8]:
df.shape

(96446, 47)

In [9]:
# Find columns with identical values in all rows
def find_columns_with_same_value(df):
    columns_with_same_value = []
    for col in df.columns:
        if df[col].nunique() == 1:
            columns_with_same_value.append(col)
    return columns_with_same_value

In [10]:
columns_same_value = find_columns_with_same_value(df)
print("Columns where all rows contain the same value:")
print(columns_same_value)

Columns where all rows contain the same value:
['examide', 'citoglipton', 'metformin-rosiglitazone']


In [11]:
df = df.drop(['citoglipton', 'examide', 'metformin-rosiglitazone'], axis = 1)

In [12]:
missing_percentages = df.apply(lambda col: (col == '?').mean() * 100 if col.dtype == object else 0)
for col, percentage in missing_percentages.items():
    if percentage > 0:
        print(f"{col}: {percentage:.2f}%") #no missings

## Feature engineering

In [13]:
# Define the comorbidity ranges (ranges bysed on https://en.wikipedia.org/wiki/List_of_ICD-9_codes)
comorbidity_ranges = {
    'infectious_parasitic_diseases': (1, 139),
    'neoplasms': (140, 239),
    'endocrine_nutritional_metabolic_immunity_disorders': (240, 279),
    'blood_diseases': (280, 289),
    'mental_disorders': (290, 319),
    'nervous_system_diseases': (320, 389),
    'circulatory_system_diseases': (390, 459),
    'respiratory_system_diseases': (460, 519),
    'digestive_system_diseases': (520, 579),
    'genitourinary_system_diseases': (580, 629),
    'pregnancy_childbirth_complications': (630, 679),
    'skin_diseases': (680, 709),
    'musculoskeletal_system_diseases': (710, 739),
    'congenital_anomalies': (740, 759),
    'perinatal_conditions': (760, 779),
    'symptoms_signs_ill_defined_conditions': (780, 799),
    'injury_poisoning': (800, 999),
    'external_causes_supplemental': ('E', 'V')
}

def is_comorbidity(code, comorbidity_ranges):
    try:
        code_int = int(code)
        for comorbidity, (start, end) in comorbidity_ranges.items():
            if start != 'E' and start != 'V' and start <= code_int <= end:
                return True
    except ValueError:
        if any(code.startswith(prefix) for prefix in ['E', 'V']):
            return True
    return False

def count_comorbidities(row, comorbidity_ranges):
    count = 0
    if is_comorbidity(row['diag_1'], comorbidity_ranges):
        count += 1
    if is_comorbidity(row['diag_2'], comorbidity_ranges):
        count += 1
    if is_comorbidity(row['diag_3'], comorbidity_ranges):
        count += 1
    return count

In [14]:
# Apply the comorbidity count function to each row
df['comorbidity_count'] = df.apply(lambda row: count_comorbidities(row, comorbidity_ranges), axis=1)

# Display the value counts for comorbidity_count
print(df['comorbidity_count'].value_counts())

comorbidity_count
3    77584
2    18210
1      642
0       10
Name: count, dtype: int64


In [15]:
# Create visit history summary
df['total_visits'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']

In [16]:
df.total_visits.max()

80

In [17]:
# # Recode admission type, discharge type and admission source into fewer categories
admission_type_mapping = {
    2: 1,
    7: 1,
    6: 5,
    8: 5
}

discharge_disposition_mapping = {
    6: 1,
    8: 1,
    9: 1,
    13: 1,
    3: 2,
    4: 2,
    5: 2,
    14: 2,
    22: 2,
    23: 2,
    24: 2,
    12: 10,
    15: 10,
    16: 10,
    17: 10,
    25: 18,
    26: 18
}

admission_source_mapping = {
    2: 1,
    3: 1,
    5: 4,
    6: 4,
    10: 4,
    22: 4,
    25: 4,
    15: 9,
    17: 9,
    20: 9,
    21: 9,
    13: 11,
    14: 11
}

# Apply mappings to re-encode the columns
df['admission_type_id'] = df['admission_type_id'].replace(admission_type_mapping)
df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(discharge_disposition_mapping)
df['admission_source_id'] = df['admission_source_id'].replace(admission_source_mapping)

In [18]:
# Time in hospital feature
df['long_stay'] = (df['time_in_hospital'] > 7).astype(int)

In [19]:
# Select medication columns
medication_start_col = 'metformin'
medication_end_col = 'metformin-pioglitazone'

# Find the range of columns for medications
medication_cols = df.loc[:, medication_start_col:medication_end_col].columns

# Add number of medication changes
for col in medication_cols:
    colname = str(col) + 'temp'
    df[colname] = df[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)
df['numchange'] = df[[str(col) + 'temp' for col in medication_cols]].sum(axis=1)

# Drop temporary columns
df.drop(columns=[str(col) + 'temp' for col in medication_cols], inplace=True)

for col in medication_cols:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)


/tmp/ipykernel_39916/271900712.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('Down', 1)
/tmp/ipykernel_39916/271900712.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace('Steady', 1)
/tmp/ipykernel_39916/271900712.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent

In [20]:
df['numchange'].value_counts()

numchange
0    70142
1    24922
2     1271
3      106
4        5
Name: count, dtype: int64

In [21]:
# Number of medications used
df['nummed'] = 0

for col in medication_cols:
    df['nummed'] = df['nummed'] + df[col]
df['nummed'].value_counts()

nummed
1    44589
0    22156
2    20901
3     7448
4     1290
5       57
6        5
Name: count, dtype: int64

### Interaction terms

In [22]:
# Define a function to convert age intervals to midpoints
def age_to_midpoint(age):
    age_mapping = {
        '[0-10)': 5,
        '[10-20)': 15,
        '[20-30)': 25,
        '[30-40)': 35,
        '[40-50)': 45,
        '[50-60)': 55,
        '[60-70)': 65,
        '[70-80)': 75,
        '[80-90)': 85,
        '[90-100)': 95
    }
    return age_mapping.get(age, 0)

# Convert age intervals to midpoints
df['age_midpoint'] = df['age'].apply(age_to_midpoint)

# Convert change to numeric
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)

/tmp/ipykernel_39916/1874684144.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['change'] = df['change'].replace('No', 0)


In [23]:
# Compute the interaction terms
interactionterms = [('num_medications','time_in_hospital'),
('num_medications','num_procedures'),
('time_in_hospital','num_lab_procedures'),
('num_medications','num_lab_procedures'),
('num_medications','number_diagnoses'),
('age_midpoint','number_diagnoses'),
('age_midpoint', 'comorbidity_count'),
('change','num_medications'),
('number_diagnoses','time_in_hospital'),
('num_medications','numchange')]

for inter in interactionterms:
    name = inter[0] + '|' + inter[1]
    df[name] = df[inter[0]] * df[inter[1]]

In [24]:
# Drop the temporary age_midpoint column
df.drop(columns=['age_midpoint'], inplace=True)

In [25]:
# Categorizing diagnosis
#9 disease categories: Circulatory, Respiratory, Digestive, Diabetes, Injury, Musculoskeletal, Genitourinary, Neoplasms, and Others.

# Creating additional columns for diagnosis
for col in ['diag_1', 'diag_2', 'diag_3']:
    df[f'level1_{col}'] = df[col]

# Replace 'V' and 'E' codes with 0
for col in ['level1_diag_1', 'level1_diag_2', 'level1_diag_3']:
    df[col] = df[col].replace({r'^V.*': 0, r'^E.*': 0}, regex=True)

# Replace '?' with -1
df.replace('?', -1, inplace=True)

# Convert columns to float
for col in ['level1_diag_1', 'level1_diag_2', 'level1_diag_3']:
    df[col] = df[col].astype(float)


In [26]:
# Define a function to classify the diagnosis levels
def classify_diag_level1(value):
    if value >= 390 and value < 460 or np.floor(value) == 785:
        return 1
    elif value >= 460 and value < 520 or np.floor(value) == 786:
        return 2
    elif value >= 520 and value < 580 or np.floor(value) == 787:
        return 3
    elif np.floor(value) == 250:
        return 4
    elif value >= 800 and value < 1000:
        return 5
    elif value >= 710 and value < 740:
        return 6
    elif value >= 580 and value < 630 or np.floor(value) == 788:
        return 7
    elif value >= 140 and value < 240:
        return 8
    else:
        return 0

In [27]:
# Apply the classification functions
df['level1_diag_1'] = df['level1_diag_1'].apply(classify_diag_level1)
df['level1_diag_2'] = df['level1_diag_2'].apply(classify_diag_level1)
df['level1_diag_3'] = df['level1_diag_3'].apply(classify_diag_level1)

In [28]:
# Drop original columns that have been encoded or aggregated
df.drop(['number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3'],
          axis=1, inplace=True)

In [29]:
df.dtypes

encounter_id                             int64
patient_nbr                              int64
race                                    object
gender                                  object
age                                     object
admission_type_id                        int64
discharge_disposition_id                 int64
admission_source_id                      int64
time_in_hospital                         int64
num_lab_procedures                       int64
num_procedures                           int64
num_medications                          int64
number_diagnoses                         int64
max_glu_serum                          float64
A1Cresult                              float64
metformin                                int64
repaglinide                              int64
nateglinide                              int64
chlorpropamide                           int64
glimepiride                              int64
acetohexamide                            int64
glipizide    

In [30]:
# Transform dtypes
i = ['encounter_id', 'patient_nbr', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',\
        'A1Cresult', 'max_glu_serum']
df[i] = df[i].astype('object')

In [31]:
# Encoding Readmission
df['readmitted'].value_counts()

readmitted
NO     50731
>30    34649
<30    11066
Name: count, dtype: int64

In [32]:
# Define replacement mapping
readmitted_mapping = {'>30': 0, '<30': 1, 'NO': 0}

# Apply replacement using the mapping
df['readmitted'] = df['readmitted'].replace(readmitted_mapping)
df['readmitted'].value_counts()

/tmp/ipykernel_39916/4251170643.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['readmitted'] = df['readmitted'].replace(readmitted_mapping)


readmitted
0    85380
1    11066
Name: count, dtype: int64

In [33]:
df.isna().sum()

encounter_id                           0
patient_nbr                            0
race                                   0
gender                                 0
age                                    0
admission_type_id                      0
discharge_disposition_id               0
admission_source_id                    0
time_in_hospital                       0
num_lab_procedures                     0
num_procedures                         0
num_medications                        0
number_diagnoses                       0
max_glu_serum                          0
A1Cresult                              0
metformin                              0
repaglinide                            0
nateglinide                            0
chlorpropamide                         0
glimepiride                            0
acetohexamide                          0
glipizide                              0
glyburide                              0
tolbutamide                            0
pioglitazone    

In [34]:
df = df.drop_duplicates(subset= ['patient_nbr'], keep = 'first')
df = df.drop(['encounter_id', 'patient_nbr'], axis=1)
df.shape

(67580, 54)

In [35]:
# Separate features and target
X = df.drop('readmitted', axis=1)
y = df['readmitted']

# Split the data into training + validation and testing sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Further split the training + validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42, shuffle=True)  # 0.25 * 0.8 = 0.2

## Preprocessing

In [36]:
# Custom transformer for Label Encoding 'age' column
class AgeLabelEncoder:
    def fit(self, X, y=None):
        self.encoder = LabelEncoder()
        self.encoder.fit(X['age'])
        return self

    def transform(self, X):
        X = X.copy()
        X['age'] = self.encoder.transform(X['age'])
        return X

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [37]:
age_label_encoder = FunctionTransformer(lambda X: AgeLabelEncoder().fit_transform(X))

# Numeric preprocessing pipeline
num_preproc = Pipeline([
    ('scaler', StandardScaler()),
])

# Categorical preprocessing pipeline (excluding 'age')
categorical_columns = [col for col in X_train.select_dtypes(include=['object']).columns if col != 'age']
cat_preproc = Pipeline([
    ('ohe', OneHotEncoder(sparse_output=False, drop="if_binary", handle_unknown='ignore')),
])
preproc = ColumnTransformer([
    ('age_label_encoder', age_label_encoder, ['age']),
    ('num_transf', num_preproc, make_column_selector(dtype_include='number')),
    ('cat_transf', cat_preproc, categorical_columns),
], verbose_feature_names_out=False).set_output(transform='pandas')

# Create a pipeline with SMOTE and preprocessing
pipe_preproc = ImbPipeline([

    ('preprocessor', preproc),
])


pipe_preproc

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('age_label_encoder',
                                                  FunctionTransformer(func=<function <lambda> at 0x7fa69b47d900>),
                                                  ['age']),
                                                 ('num_transf',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7fa708835f90>),
                                                 ('cat_transf',
                                                  Pipeline(steps=[('ohe',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['race', 'gender',
                                                   'admission_type_id',
                                                   'discharge_disposition_id',
                                                   'admission_source_id',
                                                   'max_glu_serum', 'A1Cresult',
                                                   'diabetesMed'])],
                                   verbose_feature_names_out=False))])

In [38]:
# Fit and transform the training data with preprocessing pipeline
X_train_preprocessed = pipe_preproc.fit_transform(X_train)
X_val_preprocessed = pipe_preproc.transform(X_val)
X_test_preprocessed = pipe_preproc.transform(X_test)

In [39]:
# Apply SMOTE to the preprocessed training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_preprocessed, y_train)

In [40]:
X_train_smote.head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-pioglitazone,change,comorbidity_count,total_visits,long_stay,numchange,nummed,num_medications|time_in_hospital,num_medications|num_procedures,time_in_hospital|num_lab_procedures,num_medications|num_lab_procedures,num_medications|number_diagnoses,age_midpoint|number_diagnoses,age_midpoint|comorbidity_count,change|num_medications,number_diagnoses|time_in_hospital,num_medications|numchange,level1_diag_1,level1_diag_2,level1_diag_3,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Male,admission_type_id_1,admission_type_id_3,admission_type_id_4,admission_type_id_5,discharge_disposition_id_1,discharge_disposition_id_2,discharge_disposition_id_7,discharge_disposition_id_10,discharge_disposition_id_18,discharge_disposition_id_19,discharge_disposition_id_20,discharge_disposition_id_27,discharge_disposition_id_28,admission_source_id_1,admission_source_id_4,admission_source_id_7,admission_source_id_8,admission_source_id_9,admission_source_id_11,max_glu_serum_-99.0,max_glu_serum_0.0,max_glu_serum_1.0,A1Cresult_-99.0,A1Cresult_0.0,A1Cresult_1.0,diabetesMed_Yes
0,9,2.273501,0.755757,-0.250206,0.387484,-0.168799,-0.515347,-0.113978,-0.083537,-0.032201,-0.23558,-0.004966,-0.383674,-0.351593,-0.012165,-0.285349,-0.265019,-0.053331,-0.016473,-0.007023,-0.02048,0.985116,12.236669,-0.009933,0.0,-0.004966,1.108167,-1.970106,-0.396301,2.461792,1.543340,0.872014,1.449049,-0.204320,2.199447,0.640314,0.182130,0.976909,0.070168,1.010539,1.720312,1.276569,0.649467,1.681292,1.793685,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,6,-0.782881,0.453319,-0.250206,-0.215273,-0.697655,1.940440,-0.113978,-0.083537,-0.032201,-0.23558,-0.004966,-0.383674,-0.351593,-0.012165,-0.285349,-0.265019,-0.053331,-0.016473,-0.007023,-0.02048,0.985116,-0.081722,-0.009933,0.0,-0.004966,1.108167,0.471034,-0.396301,-0.406208,-0.550175,0.872014,-0.574273,-0.311670,-0.510228,0.011022,-0.467730,-0.539644,0.162364,0.546859,-0.806368,-0.465143,-0.653924,-0.502369,-0.933225,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,7,-0.443283,1.411040,2.027986,0.025830,-0.168799,-0.515347,-0.113978,-0.083537,-0.032201,-0.23558,-0.004966,-0.383674,2.844194,-0.012165,-0.285349,-0.265019,-0.053331,-0.016473,-0.007023,-0.02048,0.985116,-0.081722,-0.009933,0.0,-0.004966,1.108167,0.471034,-0.396301,-0.406208,1.543340,0.872014,-0.350701,1.105342,0.042871,0.697523,-0.096382,0.204846,0.715542,0.732331,-0.456520,1.001562,-0.653924,-0.502369,-0.478740,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,4,-1.122479,-0.907652,2.597534,-0.094722,0.888914,-0.515347,-0.113978,-0.083537,-0.032201,-0.23558,-0.004966,-0.383674,2.844194,-0.012165,-0.285349,-0.265019,-0.053331,-0.016473,-0.007023,-0.02048,0.985116,-0.081722,-0.009933,0.0,-0.004966,1.108167,-1.970106,-0.396301,-0.406208,-0.550175,0.872014,-0.719594,1.320041,-0.911097,-0.582936,0.208655,-0.456923,-1.773758,0.639595,-0.922984,-0.465143,-0.653924,0.807827,-0.478740,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,5,0.235913,-0.050744,-0.819754,-0.938582,0.360058,-0.515347,-0.113978,-0.083537,-0.032201,-0.23558,-0.004966,2.606376,-0.351593,-0.012165,-0.285349,-0.265019,-0.053331,-0.016473,-0.007023,-0.02048,-1.015109,-0.081722,-0.009933,0.0,-0.004966,1.108167,0.471034,-0.396301,-0.406208,1.543340,-0.192037,-0.440130,-0.612248,0.027648,-0.648557,-0.732979,-0.26390

### Scores

***Precision*** = High precision indicates that when the model predicts a readmission, it is usually correct. <br>
***Recall*** =  High recall indicates that the model correctly identifies a high percentage of actual readmissions. <br>
***F1-Score*** = A higher F1-score indicates a better balance between precision and recall. <br>
***AUC-ROC*** = A higher AUC-ROC value indicates better overall performance.

## Base model (Linear Regression)

In [41]:
# Baseline Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_smote, y_train_smote)

# Predictions and evaluation
y_val_pred = log_reg.predict(X_val_preprocessed)
print("Logistic Regression Validation Performance")
print(classification_report(y_val, y_val_pred))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
logreg_score = accuracy_score(y_val, y_val_pred)

Logistic Regression Validation Performance
              precision    recall  f1-score   support

           0       0.92      0.73      0.82     12304
           1       0.11      0.35      0.17      1212

    accuracy                           0.70     13516
   macro avg       0.52      0.54      0.49     13516
weighted avg       0.85      0.70      0.76     13516

Validation Accuracy: 0.6998


## Other models

### SVC 

In [162]:
svc = SVC(random_state=42)
svc.fit(X_train_smote, y_train_smote)
y_val_pred_svc = svc.predict(X_val_preprocessed)
print("Support Vector Classifier Validation Performance")
print(classification_report(y_val, y_val_pred_svc))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_svc):.4f}")
svc_score = accuracy_score(y_val, y_val_pred_svc)

Support Vector Classifier Validation Performance
              precision    recall  f1-score   support

           0       0.92      0.89      0.90     12304
           1       0.15      0.20      0.17      1212

    accuracy                           0.83     13516
   macro avg       0.54      0.55      0.54     13516
weighted avg       0.85      0.83      0.84     13516

Validation Accuracy: 0.8275


### Random Forest

In [42]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_smote, y_train_smote)
y_val_pred_rf = rf.predict(X_val_preprocessed)
print("Random Forest Validation Performance")
print(classification_report(y_val, y_val_pred_rf))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_rf):.4f}")
rf_score = accuracy_score(y_val, y_val_pred_rf)

Random Forest Validation Performance
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     12304
           1       0.25      0.00      0.00      1212

    accuracy                           0.91     13516
   macro avg       0.58      0.50      0.48     13516
weighted avg       0.85      0.91      0.87     13516

Validation Accuracy: 0.9099


### Gradient Boosting

In [43]:
gb = GradientBoostingClassifier(random_state=42)
gb.fit(X_train_smote, y_train_smote)
y_val_pred_gb = gb.predict(X_val_preprocessed)
print("Gradient Boosting Validation Performance")
print(classification_report(y_val, y_val_pred_gb))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_gb):.4f}")
gb_score = accuracy_score(y_val, y_val_pred_gb)

Gradient Boosting Validation Performance
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     12304
           1       1.00      0.00      0.00      1212

    accuracy                           0.91     13516
   macro avg       0.96      0.50      0.48     13516
weighted avg       0.92      0.91      0.87     13516

Validation Accuracy: 0.9105


### XGBoost

In [165]:
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train_smote, y_train_smote)
y_val_pred_xgb = xgb.predict(X_val_preprocessed)
print("XGBoost Validation Performance")
print(classification_report(y_val, y_val_pred_xgb))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_xgb):.4f}")
xgb_score = accuracy_score(y_val, y_val_pred_xgb)

XGBoost Validation Performance
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     12304
           1       0.24      0.01      0.01      1212

    accuracy                           0.91     13516
   macro avg       0.58      0.50      0.48     13516
weighted avg       0.85      0.91      0.87     13516

Validation Accuracy: 0.9091


### DecisionTreeClassifier

In [166]:
dt = DecisionTreeClassifier(max_depth=28, criterion = "entropy", min_samples_split=10)
dt.fit(X_train_smote, y_train_smote)
y_val_pred_dt = dt.predict(X_val_preprocessed)
print("Decision Tree Validation Performance")
print(classification_report(y_val, y_val_pred_dt))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_dt):.4f}")
dt_score = accuracy_score(y_val, y_val_pred_dt)

Decision Tree Validation Performance
              precision    recall  f1-score   support

           0       0.91      0.91      0.91     12304
           1       0.12      0.13      0.13      1212

    accuracy                           0.84     13516
   macro avg       0.52      0.52      0.52     13516
weighted avg       0.84      0.84      0.84     13516

Validation Accuracy: 0.8408


### AdaBoostClassifier

In [167]:
ada = AdaBoostClassifier(random_state=42)
ada.fit(X_train_smote, y_train_smote)
y_val_pred_ada = ada.predict(X_val_preprocessed)
print("AdaBoost Validation Performance")
print(classification_report(y_val, y_val_pred_ada))
print(f"Validation Accuracy: {accuracy_score(y_val, y_val_pred_ada):.4f}")
ada_score = accuracy_score(y_val, y_val_pred_ada)

/Users/virginiawenger/.pyenv/versions/3.10.6/envs/hospital_readmission/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Validation Performance
              precision    recall  f1-score   support

           0       0.91      0.99      0.95     12304
           1       0.22      0.02      0.04      1212

    accuracy                           0.91     13516
   macro avg       0.56      0.51      0.50     13516
weighted avg       0.85      0.91      0.87     13516

Validation Accuracy: 0.9051


### Test set evaluation with the best model

In [168]:
# Creating a DataFrame
accuracy = {
    'Model': ['Logistic Regression', 'SVC', 'Random Forest', 'Gradient Boosting', 'XGBoost', 'Decision Tree', 'AdaBoost'],
    'Accuracy Score': [logreg_score, svc_score, rf_score, gb_score, xgb_score, dt_score, ada_score]
}

df_accuracy = pd.DataFrame(accuracy).sort_values(by='Accuracy Score', ascending=False).reset_index(drop=True)
df_accuracy

,Model,Accuracy Score
0,Gradient Boosting,0.910476
1,Random Forest,0.909885
2,XGBoost,0.909071
3,AdaBoost,0.905075
4,Decision Tree,0.840781
5,SVC,0.827464
6,Logistic Regression,0.699689


Gradient Boosting Performed best

In [44]:
best_model = gb
y_test_pred = best_model.predict(X_test_preprocessed)
print("Best Model Test Performance")
print(classification_report(y_test, y_test_pred))
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred):.4f}")

Best Model Test Performance
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     12270
           1       0.00      0.00      0.00      1246

    accuracy                           0.91     13516
   macro avg       0.45      0.50      0.48     13516
weighted avg       0.82      0.91      0.86     13516

Test Accuracy: 0.9077


In [45]:
import pickle
with open('../models/best_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)

## Improving models

### Feature Selection

In [117]:
# Feature Selection using RandomForestClassifier feature importance
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_smote, y_train_smote)

feature_importances = rf.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]

# Print feature importance
print("Feature Importances:")
for idx in sorted_idx:
    print(f"{X_train_preprocessed.columns[idx]}: {feature_importances[idx]}")

Feature Importances:
gender_Male: 0.06932768439906856
total_visits: 0.06281232175165279
discharge_disposition_id_1: 0.05100079270033313
level1_diag_3: 0.046869868864917494
discharge_disposition_id_2: 0.0431065129303967
level1_diag_2: 0.04192436198610591
admission_source_id_7: 0.039241829383113914
level1_diag_1: 0.03567544773789449
race_Caucasian: 0.03219241651610536
time_in_hospital: 0.03135782202980571
admission_source_id_1: 0.02918926416650777
A1Cresult_-99.0: 0.027898744909492342
num_procedures: 0.026373797974403534
age_midpoint|number_diagnoses: 0.026095192179587126
race_AfricanAmerican: 0.024708148035707103
age_midpoint|comorbidity_count: 0.024415948955034402
admission_type_id_1: 0.02328322111144932
time_in_hospital|num_lab_procedures: 0.022652821633198345
number_diagnoses|time_in_hospital: 0.02235194470197142
num_lab_procedures: 0.022113624395676466
num_medications|num_lab_procedures: 0.02182038329619364
number_diagnoses: 0.02111219150438721
num_medications: 0.020834020091277078


In [67]:
# # Select top k features based on importance (loop)

# # Initialize lists to store the results
# k_values = []
# validation_accuracies = []

# # Loop over different values of k
# for k in range(1, len(sorted_idx) + 1):
#     selected_features = X_train_smote.columns[sorted_idx][:k]

#     # Subset X_train_preprocessed and X_val_preprocessed with selected features
#     X_train_selected = X_train_smote[selected_features]
#     X_val_selected = X_val_preprocessed[selected_features]

#     # Retrain the SVC model with selected features
#     gb = RandomForestClassifier(random_state=42)
#     gb.fit(X_train_selected, y_train_smote)
#     y_val_pred_gb = gb.predict(X_val_selected)

#     # Evaluate the performance
#     val_accuracy = accuracy_score(y_val, y_val_pred_svc)
#     validation_accuracies.append(val_accuracy)
#     k_values.append(k)

#     #print(f"Validation Performance with top {k} features")
#     #print(classification_report(y_val, y_val_pred_gb))
#     #print(f"Validation Accuracy: {val_accuracy:.4f}")

# # Determine the best k based on the highest validation accuracy
# best_k = k_values[np.argmax(validation_accuracies)]
# best_accuracy = max(validation_accuracies)

# print(f"Best k: {best_k} with Validation Accuracy: {best_accuracy:.4f}")

Best k: 1 with Validation Accuracy: 0.7161


In [ ]:
# # Retrain the gb model with the best k features on the test set
# best_selected_features = X_train_smote.columns[sorted_idx][:best_k]
# X_train_best_selected = X_train_smote[best_selected_features]
# X_val_best_selected = X_val_preprocessed[best_selected_features]
# X_test_best_selected = X_test_preprocessed[best_selected_features]

# gb_best = RandomForestClassifier(random_state=42)
# gb_best.fit(X_train_best_selected, y_train_smote)
# y_test_pred_gb_best = gb_best.predict(X_test_best_selected)

# # Evaluate the performance on the test set
# print("Random Forest Performance with Best Selected Features")
# print(classification_report(y_test, y_test_pred_gb_best))
# print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred_gb_best):.4f}")

Feature selection did not improve my score by very much. I keep 25 features.

### Model Tuning

### Ensemble Methods

## Error Analysis

#### Confusion Matrix

In [ ]:
# # Confusion matrix for validation set
# cm_val = confusion_matrix(y_val, y_val_pred_ensemble)

# # Confusion matrix for test set
# cm_test = confusion_matrix(y_test, y_test_pred_ensemble)

# # Plot confusion matrix for validation set
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_val, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix - Validation Set')
# plt.show()

# # Plot confusion matrix for test set
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
# plt.xlabel('Predicted')
# plt.ylabel('Actual')
# plt.title('Confusion Matrix - Test Set')
# plt.show()

#### Classification Report

In [ ]:
# # Classification report for validation set
# print("Classification Report - Validation Set")
# print(classification_report(y_val, y_val_pred_ensemble))

# # Classification report for test set
# print("Classification Report - Test Set")
# print(classification_report(y_test, y_test_pred_ensemble))

#### Error Analysis on Validation Set

In [ ]:
# # Identify false positives and false negatives in validation set
# val_errors = X_val[y_val != y_val_pred_ensemble]
# val_errors['actual'] = y_val[y_val != y_val_pred_ensemble]
# val_errors['predicted'] = y_val_pred_ensemble[y_val != y_val_pred_ensemble]

# print("Validation Set Errors")
# print(val_errors.head())

####  Error Analysis on Test Set

In [ ]:
# # Identify false positives and false negatives in test set
# test_errors = X_test[y_test != y_test_pred_ensemble]
# test_errors['actual'] = y_test[y_test != y_test_pred_ensemble]
# test_errors['predicted'] = y_test_pred_ensemble[y_test != y_test_pred_ensemble]

# print("Test Set Errors")
# print(test_errors.head())

#### Visualize Feature Distributions

In [ ]:
# # Visualize feature distribution for validation set errors
# plt.figure(figsize=(14, 10))
# for i, feature in enumerate(selected_features):
#     plt.subplot(4, 2, i + 1)
#     sns.histplot(val_errors[feature], kde=True)
#     plt.title(f'Distribution of {feature} in Validation Errors')
# plt.tight_layout()
# plt.show()

# # Visualize feature distribution for test set errors
# plt.figure(figsize=(14, 10))
# for i, feature in enumerate(selected_features):
#     plt.subplot(4, 2, i + 1)
#     sns.histplot(test_errors[feature], kde=True)
#     plt.title(f'Distribution of {feature} in Test Errors')
# plt.tight_layout()
# plt.show()

## Regularization

#### Learning curves

In [ ]:
# def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
#     plt.figure(figsize=(10, 6))
#     plt.title(title)
#     if ylim is not None:
#         plt.ylim(*ylim)
#     plt.xlabel("Training examples")
#     plt.ylabel("Score")
#     train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
#     train_scores_mean = np.mean(train_scores, axis=1)
#     train_scores_std = np.std(train_scores, axis=1)
#     test_scores_mean = np.mean(test_scores, axis=1)
#     test_scores_std = np.std(test_scores, axis=1)
#     plt.grid()

#     plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
#     plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
#     plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
#     plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

#     plt.legend(loc="best")
#     return plt


In [ ]:
# # Plot learning curve for the best model
# plot_learning_curve(best_model, "Learning Curve for Best Model", X_train_selected, y_train, cv=5)
# plt.show()

#SHAP


In [46]:
import shap

/home/manuela/.pyenv/versions/3.10.6/envs/hospital_readmission/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [48]:
explainer = shap.TreeExplainer(best_model)
with open("explainer.pkl", "wb") as explainer_file:
    pickle.dump(explainer, explainer_file)